In [30]:
import pandas as pd
import numpy as np
import os
import sys
from collections import defaultdict, Counter

In [31]:
df = pd.read_csv(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\03_identify_paragraphs_containing_keywords\20210428-20210501_16.csv")
df.head(3)

,Unnamed: 0,Keyword,Paragraph,Report,PPV,TOC,Title,Subtitle,Date,Pages,Price,Contributor,Analyst,Language,Collection,Call,file,hasnumber
0,0,occ,"ricardo h. rosa - subsea 7 s.a. - cfo amy, as ...",72013054,N,Y,SUBSEA 7 S A,SUBC.OL - Event Transcript of Subsea 7 SA conf...,2021-04-29,12,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nJohn Evans Subsea ...,20210428-20210501_16.csv,True
1,1,occ,but results from january to march surpassed re...,72013024,N,Y,"HERSHA HOSPITALITY TRUST, INC.",HT.N - Event Transcript of Hersha Hospitality ...,2021-04-29,20,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAshish R. Parikh H...,20210428-20210501_16.csv,True
2,2,occ,absolute portfolio revpar sequentially improve...,72013024,N,Y,"HERSHA HOSPITALITY TRUST, INC.",HT.N - Event Transcript of Hersha Hospitality ...,2021-04-29,20,Subscription,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAshish R. Parikh H...,20210428-20210501_16.csv,True


In [32]:
correct = []
para_counts = []
count = 0

for keyword, para in zip(df["Keyword"], df["Paragraph"]):
    keyword, para = keyword.lower(), para.lower()
    try:
        start = para.index(keyword)
        end = start + len(keyword)
    except:
        count += 1
        continue
    
    if start == 0:
        full_keyword = keyword + " "
    elif end == len(para) - 1:
        full_keyword = " " + keyword
    else:
        full_keyword = " " + keyword + " "
    
    
    correct.append(full_keyword in para)
    num_counts = para.count(full_keyword)
    para_counts.append(num_counts)

df["Correct"] = correct
df["NumberOfOccurences"] = para_counts
final_df = df[df["Correct"] == 1]

In [33]:
df.shape

(145, 20)

In [34]:
final_df.shape

(35, 20)

In [36]:
# Get a list of keywords that each paragraph contains
para_dict = defaultdict(list)
for para in final_df["Paragraph"].unique():
    # Get all occurences of that particular paragraph
    subset = final_df[final_df["Paragraph"] == para]
    # Create a list of all keywords that appeared in that paragraph
    para_dict[para].append(list(subset["Keyword"]))
final_df["List_Keywords"] = final_df["Paragraph"].apply(lambda x: list(set((para_dict[x])[0])))
#final_df = final_df.drop_duplicates("Para")
final_df.head(5)

C:\Users\jasonjia\AppData\Local\Temp\ipykernel_9564\1684838529.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["List_Keywords"] = final_df["Paragraph"].apply(lambda x: list(set((para_dict[x])[0])))


,Unnamed: 0,Keyword,Paragraph,Report,PPV,TOC,Title,Subtitle,Date,Pages,...,Contributor,Analyst,Language,Collection,Call,file,hasnumber,Correct,NumberOfOccurences,List_Keywords
27,27,cost of capital,and then how would you balance putting some mo...,72013024,N,Y,"HERSHA HOSPITALITY TRUST, INC.",HT.N - Event Transcript of Hersha Hospitality ...,2021-04-29,20,...,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAshish R. Parikh H...,20210428-20210501_16.csv,False,True,1,[cost of capital]
28,28,cost of capital,so it's very difficult for us to consider issu...,72013024,N,Y,"HERSHA HOSPITALITY TRUST, INC.",HT.N - Event Transcript of Hersha Hospitality ...,2021-04-29,20,...,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAshish R. Parikh H...,20210428-20210501_16.csv,False,True,1,[cost of capital]
29,29,cost of capital,"we think that the back half of this year 2022,...",72013024,N,Y,"HERSHA HOSPITALITY TRUST, INC.",HT.N - Event Transcript of Hersha Hospitality ...,2021-04-29,20,...,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nAshish R. Parikh H...,20210428-20210501_16.csv,True,True,1,[cost of capital]
34,34,interest rate,after finalizing our green financing framework...,72013040,N,Y,ALLIED PROPERTIES REIT,AP_u.TO - Event Transcript of Allied Propertie...,2021-04-29,19,...,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nCecilia Catalina W...,20210428-20210501_16.csv,True,True,1,[interest rate]
48,48,cost of capital,so the potential to increase the economic prod...,72013040,N,Y,ALLIED PROPERTIES REIT,AP_u.TO - Event Transcript of Allied Propertie...,2021-04-29,19,...,THOMSON REUTERS STREETEVENTS,ANON,English,INV,\n CORPORATE PARTICIPANTS\nCecilia Catalina W...,20210428-20210501_16.csv,True,True,1,[cost of capital]


In [ ]:
final_df.to_csv("/project/kh_mercury_1/conference_call/output/04_keyword_identification/04.4_groups_keyword_test1/Duplicative_Amended_Correct_No_IR.csv")